# junk

In [ ]:
%%manim -ql -v WARNING --disable_caching GridOrbitAnimation

class GridOrbitAnimation(Scene):
    """Generates a grid of points coloured by location which then evolve discretly

    Args:
    !!need to be set outside of scene
        #axis params
        x_high = 1.5
        x_low= -x_high
        y_high = x_high
        y_low= -x_high
        xtick = int(np.round((x_high-x_low)/3,0))
        ytick = int(np.round((y_high-y_low)/3,0))
        
        steps = 80 #how many time steps to show
        display_axes  = False
        func = rotationM #/figure1chaos #the orbitfunction(x0)
        #animate it
        %manim -ql -v WARNING --disable_caching GridOrbitAnimation
    """
    def construct(self):
        viridis_cmap = matplotlib.colormaps['viridis'] #get color map
        def point_to_color(x, y):
            normed_value = norm(x+y) # Use both coordinates for color mapping
            color = viridis_cmap(normed_value)
            return list(color[:3]) #get rid of alpha channel
        #create axis even if not shown important for scaling
        x_range = y_range= [ x_low,x_high, xtick]
        y_range= [y_low, y_high, ytick]
        axis_length = 7
        axes = Axes(
            x_range=x_range,
            y_range=y_range,
            x_length=axis_length,
            y_length=axis_length,
            axis_config={"include_numbers": True}  # Display numbers on axes
        )
        if display_axes: self.play(Create(axes))

        #create grid of dots
        weights, orbits = twoDbifrucation()
        dots_coord = [orbit[0] for orbit in orbits]

        #create colors based on location
        # normarray = x_coords+y_coords #for colors
        # norm = matplotlib.colors.Normalize(vmin=normarray.min(), vmax=normarray.max())
        # colors = [ManimColor.from_rgb(point_to_color(x, y)) for x in x_coords for y in x_coords]
        
        
        #calculate orbits for each dot
        orbits3D = []
        for orbit in orbits:
            
            #need to add 0 z coord for each coordinate
            orbits3D.append(np.concatenate([orbit, np.zeros((1,orbit.shape[1] ))]))
        
        manim_orbits = [axes.c2p(*(orbit)) for orbit3 in orbits3D] #scale onto manim axes
        #create actual dots at starting positions
        dots = VGroup(*[Dot(orbit0,radius=0.05) for orbit0 in orbits3D[0].T ])
        self.add(dots)
        self.wait(2)
        #move dots about to create animation
        num_steps = 30#orbits[0].shape[1]
        for t in range(num_steps):
            if t == 1: self.wait(0.5)
            for i, dot in enumerate(dots):
                #use c2p again to make sure correclty scaled
                dot.move_to(orbits3D[t][:,i])
            self.wait(0.2)  # pause to visualize each time step

        orbit_dots = always_redraw(lambda: VGroup(
            *[Dot(point=point, radius=0.05, color=BLUE) for point in generate_orbit(mu_tracker.get_value())]
        ))
    


In [ ]:
%%manim -ql -v WARNING --disable_caching GridOrbitAnimation

class GridOrbitAnimation(Scene):
    """Generates a grid of points coloured by location which then evolve discretly

    Args:
    !!need to be set outside of scene
        #axis params
        x_high = 1.5
        x_low= -x_high
        y_high = x_high
        y_low= -x_high
        xtick = int(np.round((x_high-x_low)/3,0))
        ytick = int(np.round((y_high-y_low)/3,0))
        
        steps = 80 #how many time steps to show
        display_axes  = False
        func = rotationM #/figure1chaos #the orbitfunction(x0)
        #animate it
        %manim -ql -v WARNING --disable_caching GridOrbitAnimation
    """
    def construct(self):
        viridis_cmap = matplotlib.colormaps['viridis'] #get color map
        def point_to_color(x, y):
            normed_value = norm(x+y) # Use both coordinates for color mapping
            color = viridis_cmap(normed_value)
            return list(color[:3]) #get rid of alpha channel
        #create axis even if not shown important for scaling
        x_range = y_range= [ x_low,x_high, xtick]
        y_range= [y_low, y_high, ytick]
        axis_length = 7
        axes = Axes(
            x_range=x_range,
            y_range=y_range,
            x_length=axis_length,
            y_length=axis_length,
            axis_config={"include_numbers": True}  # Display numbers on axes
        )
        if display_axes: self.play(Create(axes))

        #create grid of dots
        weights, orbits = twoDbifrucation()
        dots_coord = [orbit[0] for orbit in orbits]

        #create colors based on location
        # normarray = x_coords+y_coords #for colors
        # norm = matplotlib.colors.Normalize(vmin=normarray.min(), vmax=normarray.max())
        # colors = [ManimColor.from_rgb(point_to_color(x, y)) for x in x_coords for y in x_coords]
        
        
        #calculate orbits for each dot
        orbits3D = []
        for orbit in orbits:
            
            #need to add 0 z coord for each coordinate
            orbits3D.append(np.concatenate([orbit, np.zeros((1,orbit.shape[1] ))]))
        
        manim_orbits = [axes.c2p(*(orbit)) for orbit3 in orbits3D] #scale onto manim axes
        #create actual dots at starting positions
        dots = VGroup(*[Dot(orbit0,radius=0.05) for orbit0 in orbits3D[0].T ])
        self.add(dots)
        self.wait(2)
        #move dots about to create animation
        num_steps = 30#orbits[0].shape[1]
        for t in range(num_steps):
            if t == 1: self.wait(0.5)
            for i, dot in enumerate(dots):
                #use c2p again to make sure correclty scaled
                dot.move_to(orbits3D[t][:,i])
            self.wait(0.2)  # pause to visualize each time step
    


In [ ]:
def twoDbifrucation():

    x0 = -np.array([-0.1,0.1])

    theta = 3
    W = lambda alpha : np.array([[np.cos(theta),np.sin(theta)],
                        [-np.sin(theta),np.cos(theta)]]) * alpha
    b = -np.array([-0.,0.])
    paramspace = np.linspace(1, 2, 20)

    x1_values = []
    weights = []
    steps = 200
    transient_steps = 20

    for param in paramspace:
        orbit = neuron_2_discrete(x0, W(param), b, steps,transient_steps).T
        
        # Only take the last few steps to plot bifurcation points
        x1_values.append(orbit)
        weights.append([param] * len(orbit))

    return weights, x1_values
    # plt.scatter(x1_values[0][:,0], x1_values[0][:,1], s=0.5)




In [ ]:
%%manim -ql -v WARNING --disable_caching BifrucationPlotter2D

class BifrucationPlotter2D(Scene):
    def construct(self):
        viridis_cmap = matplotlib.colormaps['viridis'] #get color map
        def point_to_color(x, y):
            normed_value = norm(x+y) # Use both coordinates for color mapping
            color = viridis_cmap(normed_value)
            return list(color[:3]) #get rid of alpha channel
        
        x_range = y_range= [ x_low,x_high, xtick]
        y_range= [y_low, y_high, ytick]
        axis_length = 7
        axes = Axes(
            x_range=x_range,
            y_range=y_range,
            x_length=axis_length,
            y_length=axis_length,
            axis_config={"include_numbers": True}  # Display numbers on axes
        )
        if display_axes: self.play(Create(axes))

        grid_size = 5  # number of points along each grid size
        x_coords =  y_coords = np.linspace(x_range[0], x_range[1], grid_size)
        y_coords = np.linspace(y_range[0], y_range[1], grid_size)
        dots_coord = [np.array([x,y]) for x in x_coords for y in x_coords]
        normarray = x_coords+y_coords #for colors
        norm = matplotlib.colors.Normalize(vmin=normarray.min(), vmax=normarray.max())
        colors = [ManimColor.from_rgb(point_to_color(x, y)) for x in x_coords for y in x_coords]
        
        # Parameter setup
        param_tracker = ValueTracker(param_range[0])
        
        # Plane and labels
        #plane = NumberPlane(x_range=[-2, 2], y_range=[-2, 2], background_line_style={"stroke_opacity": 0.5})
        #self.add(plane)
        # Dot to follow orbit
        
        orbit_dots = VGroup(
                *[Dot(point=axes.c2p(*point), radius=0.05, color=colors[i]) for i,point in enumerate(dots_coord)]
            )
        self.add(orbit_dots)
        self.wait(2)
        # Orbit updater function
        
        orbit_dotsall = []
        # Dot to follow orbit
        for i,x0 in enumerate(dots_coord):
            color = colors[i]
            orbit_dots = always_redraw(lambda i=i: VGroup(
                *[Dot(point=axes.c2p(*point), radius=0.05) for point in func(param_tracker.get_value(),x0)]
            ))
            orbit_dotsall.append(orbit_dots)

        # Display orbit dots and label
        self.add(*orbit_dotsall)
        self.play(param_tracker.animate.set_value(param_range[1]), rate_func=linear, run_time=run_time)  # Adjust final mu value
        self.wait(1)